In [1]:
import inference.InferCode_Inference as II
import data_process.self_supervised.data_reader as data_reader
import xml.etree.ElementTree as ET
import numpy as np
import time
import tqdm
import copy
import math

token_dict_path = "/home/stanley/Desktop/dictionaries/v2_dic/token2id.json"
type_dict_path = "/home/stanley/Desktop/dictionaries/v2_dic/type2id.json"
subtree_count = 633916 # 633916 for the SS-PTM v2
dimension = 64

# code examples to get code vector
file_path = "/home/stanley/Desktop/test_20k"
# file_path = "/home/stanley/Desktop/test_dataset_100k"

In [15]:

# construct a tree that has 'percentage(e.g., 0.1 or 10%)' less number of nodes
def reduce(tree, percentage):
    reduced_tree = copy.deepcopy(tree)
    node_count = len(reduced_tree.getroot().findall(".//*"))
    threshold = math.ceil(node_count * percentage)
    delete_e = []
    difference = node_count

    for e in reduced_tree.iter():
        num_child = len(e.findall(".//*"))
        if abs(num_child - threshold) < difference :
            delete_e = e
            difference = abs(num_child - threshold)
    delete_e.clear()
    delete_e.tag = "unknown_type"
    delete_e.text = "unknown_token"

    #return 1.0 - len(reduced_tree.getroot().findall(".//*"))/node_count
    return reduced_tree


In [16]:
dataset = data_reader.Data_Reader(file_path).processed_dataset

Data Processing : 100%|██████████| 20705/20705 [00:55<00:00, 375.34it/s]


In [17]:
def collect_file_paths(path):
        '''
        collect individual xml file path and return those as a list
        '''
        file_paths = []
        for root, directories, filenames in os.walk(path):
            for filename in filenames:
                file_paths.append(os.path.join(root,filename))
        return file_paths

def get_node_count(tree):
    return len(tree.getroot().findall(".//*"))

In [18]:
paths = collect_file_paths(file_path)
full_tree = []
tree_10 = []
tree_30 = []

len_full = [] # the number of nodes for each original tree
len_10 = [] # the number of nodes for each tree with 10% less nodes
len_30 = [] # the number of nodes for each tree with 30% less nodes

for idx, path in tqdm.tqdm(enumerate(paths)):
    original_tree = ET.parse(path)
    full_tree.append(copy.deepcopy(original_tree))
    tree_10.append(reduce(original_tree, 0.1))
    tree_30.append(reduce(original_tree, 0.3))
    len_full.append(get_node_count(original_tree))
    len_10.append(len(tree_10[idx].getroot().findall(".//*")))
    len_30.append(len(tree_30[idx].getroot().findall(".//*")))

20705it [02:08, 160.69it/s]


In [33]:
model_direct = "/home/stanley/Desktop/one_model"
model_paths = collect_file_paths(model_direct)
for model_path in model_paths:
    inferencer = II.InferCode_Inference(model_path, token_dict_path, type_dict_path, subtree_count, dimension)

    positive_count = 0
    total = len(paths)
    difference_threshold = 0.1 # 10%
    negative_count = 0
    total_diff = 0
    for idx, path in tqdm.tqdm(enumerate(paths)):
        
        difference = len_10[idx]/len_full[idx] - len_30[idx]/len_full[idx]

        if difference < difference_threshold: # remove examples that have a small difference in removed nummber of nodes
            total -= 1
        else:

            code_vector = inferencer.code2vec(copy.deepcopy(full_tree[idx]), tree = True)
            code_vector_10 = inferencer.code2vec(copy.deepcopy(tree_10[idx]), tree = True)
            code_vector_30 = inferencer.code2vec(copy.deepcopy(tree_30[idx]), tree = True)
            
            # if cos(code_vector, code_vector_10) > cos(code_vector, code_vector_30):
            if np.linalg.norm(code_vector-code_vector_10) < np.linalg.norm(code_vector-code_vector_30):
                positive_count += 1
            else:
                # for the negative ones, calculate the average distance
                negative_count += 1
                total_diff += difference
    print(model_path, " has accuracy :", positive_count / total, " The total number is ", total)
    print("average difference for negative pairs", total_diff/negative_count)
    # SS-PTM-v1 with momentum, epoch 59, accuracy: 88%
    # SS-PTM-v2-pos with momentum, epoch 1, accuracy: 88.6%
    # SS-PTM-v2 with momentum, epoch 11, accuracy: 87.1%

20705it [06:58, 49.53it/s]
/home/stanley/Desktop/one_model/epoch_49.pkl  has accuracy : 0.80574456218628  The total number is  17930
average difference for negative pairs 0.18458842525254043
20705it [06:28, 53.35it/s]
/home/stanley/Desktop/one_model/epoch_50.pkl  has accuracy : 0.807585052983826  The total number is  17930
average difference for negative pairs 0.18401247672851007
20705it [07:39, 45.05it/s]
/home/stanley/Desktop/one_model/epoch_48.pkl  has accuracy : 0.803792526491913  The total number is  17930
average difference for negative pairs 0.18496711345668812
20705it [06:53, 50.07it/s]
/home/stanley/Desktop/one_model/epoch_45.pkl  has accuracy : 0.7998884551031791  The total number is  17930
average difference for negative pairs 0.1863617559393551
20705it [06:26, 53.54it/s]
/home/stanley/Desktop/one_model/epoch_46.pkl  has accuracy : 0.8034578918014501  The total number is  17930
average difference for negative pairs 0.18656137505755804
20705it [07:23, 46.71it/s]/home/stanley/

In [42]:
# generate code vector examples
model2 = II.InferCode_Inference("/home/stanley/Desktop/SS-PTM-v2-0.01/epoch_11.pkl", token_dict_path, type_dict_path, subtree_count, dimension)
bsort1 = "/home/stanley/Desktop/code1.xml"
bsort2 = "/home/stanley/Desktop/code2.xml"
bsort3 = "/home/stanley/Desktop/code3.xml"

def distance(vec1, vec2):
    return np.linalg.norm(vec1-vec2)

def cos(x, y):
    return np.dot(x,y) / (np.sqrt(np.dot(x,x)) * np.sqrt(np.dot(y,y)))

def dot(x, y):
    return np.dot(x, y)

vec_1 = model2.code2vec(bsort1)
vec_2 = model2.code2vec(bsort2)
vec_3 = model2.code2vec(bsort3)

d12 = distance(vec_1, vec_2)
d13 = distance(vec_1, vec_3)
d23 = distance(vec_2, vec_3)

print(vec_3)

print("distance between function 1 and 2", d12)
print("distance between function 1 and 3", d13)
print("distance between function 2 and 3", d23)


[ 1.         -0.07628536  0.9619528   1.         -0.07628536  1.
  0.87018216 -0.8740815   0.87018216  0.87018216  1.          0.18330956
  0.87018216 -0.07628536 -0.91525054  1.          0.16493416 -1.
  0.99892366  0.90068054  1.          0.99892366  1.         -1.
  0.99892366  0.19153321  1.          0.9113512   1.          0.9396781
  1.          0.88855755  1.          0.97720635  1.          1.
  1.          1.          1.          0.98543     0.9396781   0.87018216
  1.          1.         -1.          0.91525054  0.98435366  1.
  0.9950243  -0.9305041   1.          1.         -0.9113512   0.18330956
  1.          0.07628536  1.          1.          1.          1.
  0.97720635  0.07628536 -1.         -0.19153321]
distance between function 1 and 2 45.353565
distance between function 1 and 3 48.71519
distance between function 2 and 3 46.967293
